In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import csv
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels.tsa import stattools
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import acf, pacf
import os
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# sess = tf.Session(config=config)

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.layers import LSTM, RNN, GRU, Dropout
from math import sqrt
from matplotlib import pyplot
from numpy import array
from sklearn.preprocessing import MinMaxScaler
os.environ['CUDA_VISIBLE_DEVICES']=''
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 6)

from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [ ]:
data = pd.read_csv('lstm_data.csv', header=None, names=['datetime', 'load'], index_col=[0], parse_dates=[0], infer_datetime_format=True)
df = pd.DataFrame(columns=['time'] + list(map(str, range(int(data.shape[0] / 288)))))
for idx, time in enumerate(sorted(set(data.index.time))):
    df.loc[idx] = [time.strftime(format='%H:%M:%S')] + list(data.at_time(time)['load'].values)
df.index = df['time']
df = df.drop('time', 1)
dt_df = df.diff(1, axis=1)
dt_df = dt_df.dropna(axis=1)
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler = scaler.fit(dt_df.values.reshape(-1, 1))
dt_df = scaler.transform(dt_df)  # dt_df is now a numpy array
split_idx = int(len(dt_df) * 0.8)
train, val = dt_df[:split_idx, :], dt_df[split_idx:, :]

In [ ]:
def prepare_data(data, nlags):
    '''prepares data for LSTM model, x=last nlags values, y=(nlags+1)'th value'''
    data_x, data_y = [], []
    for i in range(data.shape[0]):
        for j in range(0, data.shape[1]-nlags):
            data_x.append(data[i, j:j+nlags])
            data_y.append(data[i, j+nlags])
    data_x = np.array(data_x)
    data_y = np.array(data_y).reshape(-1, 1)
    return data_x, data_y

In [ ]:
nlags = 10
train_x, train_y = prepare_data(train, nlags)
val_x, val_y = prepare_data(val, nlags)
train_x = train_x.reshape(train_x.shape[0], 1, nlags)
val_x = val_x.reshape(val_x.shape[0], 1, nlags)
df_last_nlags_plus_one = df.loc[:, df.columns[-nlags-1:]]  # slice last nlags+1 days from df, will be used in prediction of yesterday's data
dt_df_last_nlags = df_last_nlags_plus_one.diff(1, axis=1).dropna(axis=1)  #taking last 21 days, differencing and dropping the nan value
dt_df_last_nlags = scaler.transform(dt_df_last_nlags)  # df is now a numpy array
X = dt_df_last_nlags.reshape(dt_df_last_nlags.shape[0], 1, nlags)  # nlags=20
load = np.load('25nov.npy')

## Model training

In [116]:
del model
batch_size = 1
model = Sequential()
model.add(GRU(1, batch_input_shape=(batch_size, train_x.shape[1], train_x.shape[2]), stateful=True, return_sequences=True))
model.add(GRU(1, stateful=True))

model.add(Dense(train_y.shape[1]))

model.compile(loss='mean_squared_error', optimizer='sgd')
es = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='auto')

In [117]:
# history = model.fit(train_x, train_y, epochs=20, batch_size=batch_size, verbose=1, validation_data=(val_x, val_y), shuffle=False, callbacks=[es])


In [118]:
# fit network
early_stopping_counter = 0
last_loss = np.inf
for i in range(10):
    history = model.fit(train_x, train_y, epochs=1, batch_size=batch_size, verbose=1, validation_data=(val_x, val_y), shuffle=False)
    model.reset_states()
    val_loss = history.history['val_loss'][0]
    if val_loss > last_loss:
        early_stopping_counter += 1
    else:    
        early_stopping_counter = 0
    last_loss = val_loss
    if early_stopping_counter == 3:
#         break
        pass

Train on 11270 samples, validate on 2842 samples
Epoch 1/1
11270/11270 [==============================] - 54s 5ms/step - loss: 0.0378 - val_loss: 0.0231
Train on 11270 samples, validate on 2842 samples
Epoch 1/1
11270/11270 [==============================] - 47s 4ms/step - loss: 0.0373 - val_loss: 0.0229
Train on 11270 samples, validate on 2842 samples
Epoch 1/1
11270/11270 [==============================] - 45s 4ms/step - loss: 0.0364 - val_loss: 0.0227
Train on 11270 samples, validate on 2842 samples
Epoch 1/1
11270/11270 [==============================] - 50s 4ms/step - loss: 0.0347 - val_loss: 0.0226
Train on 11270 samples, validate on 2842 samples
Epoch 1/1
11270/11270 [==============================] - 48s 4ms/step - loss: 0.0321 - val_loss: 0.0227
Train on 11270 samples, validate on 2842 samples
Epoch 1/1
11270/11270 [==============================] - 56s 5ms/step - loss: 0.0292 - val_loss: 0.0226
Train on 11270 samples, validate on 2842 samples
Epoch 1/1
11270/11270 [==========

#### Prediction

In [ ]:
val_y_pred = model.predict(val_x, batch_size=batch_size, verbose=1)
inverted_val_y = scaler.inverse_transform(val_y)
inverted_val_y_pred = scaler.inverse_transform(val_y_pred)
Y = model.predict(X, batch_size=batch_size)  # predict for today's values
inv_Y = scaler.inverse_transform(Y)  # invert to detrended values' scale
rescaled_Y = [x+y for x, y in zip(inv_Y[:, 0], df.iloc[:, -1])]  # last day's values added to inv_Y to get it to original scale


In [ ]:
val_RMSE = np.sqrt(np.sum(np.square(np.array(inverted_val_y_pred) - np.array(inverted_val_y))) / len(inverted_val_y)) 
RMSE = np.sqrt(np.sum(np.square(np.array(rescaled_Y) - load)) / len(load)) 
print(val_RMSE, RMSE)

In [ ]:
plt.plot(load)
plt.plot(rescaled_Y)
plt.legend(['load', 'pred'])

In [119]:
rescaled_Y

[1715.823553466797,
 1708.9954107666015,
 1699.1856634521484,
 1657.3427954101562,
 1651.6024182128906,
 1647.3158380126954,
 1628.859617919922,
 1596.32791015625,
 1594.3380261230468,
 1593.8417608642578,
 1571.0069506835937,
 1560.4428381347657,
 1557.357671508789,
 1528.4894995117188,
 1523.5865301513672,
 1521.875785522461,
 1506.042491455078,
 1504.019232788086,
 1491.3736163330077,
 1487.476370239258,
 1478.1452545166017,
 1482.7583984375,
 1483.5092010498047,
 1481.1731085205079,
 1474.6028979492187,
 1464.0640905761718,
 1449.824649658203,
 1449.9458660888672,
 1446.4299926757812,
 1446.852310180664,
 1446.063759765625,
 1444.1601654052733,
 1436.735596923828,
 1440.4698449707032,
 1425.2521459960938,
 1426.4672717285157,
 1420.3052972412108,
 1422.7793481445312,
 1413.5216162109375,
 1418.168145751953,
 1416.3054956054686,
 1408.0576611328124,
 1418.4973364257812,
 1421.048167114258,
 1423.9135736083983,
 1428.442795715332,
 1436.8973370361327,
 1428.7753771972657,
 1427.98025

In [139]:
!head predictions/27-11-2018.csv

time,load
00:00:00,1715.823553466797
00:05:00,1708.9954107666015
00:10:00,1699.1856634521484
00:15:00,1657.3427954101562
00:20:00,1651.6024182128906
00:25:00,1647.3158380126954
00:30:00,1628.859617919922
00:35:00,1596.32791015625
00:40:00,1594.3380261230468


In [122]:
list(df.index)

['00:00:00',
 '00:05:00',
 '00:10:00',
 '00:15:00',
 '00:20:00',
 '00:25:00',
 '00:30:00',
 '00:35:00',
 '00:40:00',
 '00:45:00',
 '00:50:00',
 '00:55:00',
 '01:00:00',
 '01:05:00',
 '01:10:00',
 '01:15:00',
 '01:20:00',
 '01:25:00',
 '01:30:00',
 '01:35:00',
 '01:40:00',
 '01:45:00',
 '01:50:00',
 '01:55:00',
 '02:00:00',
 '02:05:00',
 '02:10:00',
 '02:15:00',
 '02:20:00',
 '02:25:00',
 '02:30:00',
 '02:35:00',
 '02:40:00',
 '02:45:00',
 '02:50:00',
 '02:55:00',
 '03:00:00',
 '03:05:00',
 '03:10:00',
 '03:15:00',
 '03:20:00',
 '03:25:00',
 '03:30:00',
 '03:35:00',
 '03:40:00',
 '03:45:00',
 '03:50:00',
 '03:55:00',
 '04:00:00',
 '04:05:00',
 '04:10:00',
 '04:15:00',
 '04:20:00',
 '04:25:00',
 '04:30:00',
 '04:35:00',
 '04:40:00',
 '04:45:00',
 '04:50:00',
 '04:55:00',
 '05:00:00',
 '05:05:00',
 '05:10:00',
 '05:15:00',
 '05:20:00',
 '05:25:00',
 '05:30:00',
 '05:35:00',
 '05:40:00',
 '05:45:00',
 '05:50:00',
 '05:55:00',
 '06:00:00',
 '06:05:00',
 '06:10:00',
 '06:15:00',
 '06:20:00',

In [40]:
# !jupyter nbconvert --to script LSTM-v2.ipynb

[NbConvertApp] Converting notebook LSTM-v2.ipynb to script
[NbConvertApp] Writing 9421 bytes to LSTM-v2.py


In [41]:
# ## get today's load

# def get_load_data(date):
#     load=[]
#     url = 'http://www.delhisldc.org/Loaddata.aspx?mode='
#     print('Scraping ' + date, end=' ')
#     resp = requests.get(url + date) # send a get request to the url, get response
#     soup = BeautifulSoup(resp.text, 'lxml') # Yummy HTML soup
#     table = soup.find('table', {'id':'ContentPlaceHolder3_DGGridAv'}) # get the table from html
#     trs = table.findAll('tr') # extract all rows of the table
#     if len(trs[1:])==288: # no need to create csv file, if there's no data
#         with open('monthdata.csv', 'a') as f:  #'a' makes sure the values are appended at the end of the already existing file
            
#             for tr in trs[1:]:
#                 time, delhi = tr.findChildren('font')[:2]
#                 load.append(delhi.text)
#     if len(trs[1:]) != 288:
#         print('Some of the load values are missing..')
#     else:
#         print('Done')
#     return load

# yesterday = datetime.today() - timedelta(1)
# yesterday = yesterday.strftime('%d/%m/%Y')
# load = get_load_data(yesterday)
# load = [float(x) for x in load]
# np.save('25nov.npy', load)